In [ ]:
def SL(output_folder_name, num_days, seed_number, patients_included,function_name,mins_before_predict_list, window_slide_list, epochs, batch_size, learning_rate,monitor):    
    output_folder_results = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(function_name)+'/results'
    os.makedirs(output_folder_results, exist_ok=True) 
    output_folder_models = str(output_folder_name)+"/"+str(num_days)+'_days_data_'+str(function_name)+'/models'
    os.makedirs(output_folder_models, exist_ok=True) 
    results_df = pd.DataFrame(columns=['File', 'mins_before_predict','window_slide','RMSE'])   
    for i in range(len(patients_included)):
        random.seed(seed_number) 
        np.random.seed(seed_number)
        tf.random.set_seed(seed_number)
        #######get X and y####################
        file = patients_included[i]
        directory = os.getcwd()
        file_path = os.path.join(directory, file)
        print(file_path)
        df = pd.read_csv(file_path)
        df = df[0:480*num_days] ###########use 5 days datat only
                    
        df_training = df.iloc[:int(480*(num_days-2))]
        df_validation = df.iloc[int(480*(num_days-2)):int(480*(num_days-1))]
        df_testing = df.iloc[int(480*(num_days-1)):480*num_days]


        # Extract the input features and target variable
        X_training = df_training[['insulin', 'CHO', 'CGM']]
        y_training = df_training['CGM']

        X_validation = df_validation[['insulin', 'CHO', 'CGM']]
        y_validation = df_validation['CGM']

        X_testing = df_testing[['insulin', 'CHO', 'CGM']]
        y_testing = df_testing['CGM']
        # Convert the DataFrame to numpy arrays
        X_training= X_training.to_numpy()
        y_training = y_training.to_numpy()

        X_validation= X_validation.to_numpy()
        y_validation = y_validation.to_numpy()

        X_testing= X_testing.to_numpy()
        y_testing = y_testing.to_numpy()
            
            
        for mins_before_predict in mins_before_predict_list:
            print("mins_before_predict: "+str(mins_before_predict))
            timesteps = int(mins_before_predict/3)
            
            for window_slide in window_slide_list:
                print("window slide: "+str(window_slide))
                RMSE = []
                
                
                # Create input sequences and corresponding target values
                X_sequences_training = []
                y_targets_training = []

                X_sequences_validation = []
                y_targets_validation = []

                X_sequences_testing = []
                y_targets_testing = []
            
                for i in range(0, len(X_training) - timesteps-window_slide):
                    X_sequences_training.append(X_training[i:i+window_slide])
                    y_targets_training.append(y_training[i+timesteps+window_slide-1])

                # Convert sequences and targets to numpy arrays
                X_sequences_training = np.array(X_sequences_training)
                y_targets_training = np.array(y_targets_training)

                for i in range(0, len(X_validation) - timesteps-window_slide):
                    X_sequences_validation.append(X_validation[i:i+window_slide])
                    y_targets_validation.append(y_validation[i+timesteps+window_slide-1])

                # Convert sequences and targets to numpy arrays
                X_sequences_validation = np.array(X_sequences_validation)
                y_targets_validation = np.array(y_targets_validation)


                for i in range(0, len(X_testing) - timesteps-window_slide):
                    X_sequences_testing.append(X_testing[i:i+window_slide])
                    y_targets_testing.append(y_testing[i+timesteps+window_slide-1])

                # Convert sequences and targets to numpy arrays
                X_sequences_testing = np.array(X_sequences_testing)
                y_targets_testing = np.array(y_targets_testing)


                y_test, y_pred, RMSE = deep_learning_analysis(output_folder_name, seed_number,  function_name, mins_before_predict, window_slide, epochs, batch_size, learning_rate,monitor, neuron_nums,X_sequences_training, y_targets_training, X_sequences_validation, y_targets_validation, X_sequences_testing, y_targets_testing)
            
                model = load_model(os.path.join(output_folder_models, 'best_model_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+'.h5'))
                file_name_global = "RNN_"+str(file)+'_mins_before_predict_'+str(mins_before_predict)+'_window_slide_'+str(window_slide)+".h5"
                model.save(os.path.join(output_folder_models, file_name_global))

                new_data = {'File': file, 'mins_before_predict': mins_before_predict,'window_slide': window_slide, 'RMSE': RMSE}
                new_df = pd.DataFrame([new_data])
                # Concatenate the existing results_df DataFrame with the new DataFrame
                results_df = pd.concat([results_df, new_df], ignore_index=True)
    
    
    output_filename = os.path.join(output_folder_results, str(function_name) + '.xlsx')
    results_df.to_excel(output_filename, index=False)    